In [ ]:
# load audio file
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import soundfile as sf

audio = './chtit jen malo doprovod.wav'
data, fs = sf.read(audio)

# only take the first 5 seconds
data = data[:5*fs]


# transfer function: y[n] = -c*x[n] + d(c-1)x[n-1] + x[n-1] - d(1-c)*y[n-1] + c*y[n-2]
fc = 2000 # zkusit 650
fb = 500 # zkusit 300

c = (math.tan(math.pi*fb/fs) - 1) / (math.tan(math.pi*fb/fs) + 1)
d = -math.cos(2*math.pi*fc/fs)

#c = (math.tan(math.pi*fc/fs) - 1) / (math.tan(math.pi*fc/fs) + 1)
#d = -math.cos(2*math.pi*fc/fs)

B = [-c, d*(c-1), 1]
A = [1, -d*(1-c), c]


all_pass = signal.lfilter(B, A, data)

# create band pass
# H(z) = 1/2 * (1-filtered)

# substract filtered from original signal
band_pass = data - all_pass

G = 20
V0 = 10**(G/20)
H0 = V0 - 1

y_peakfilter = 1 + (H0/2) * (1-band_pass)


sf.write('chtit_jen_malo_zpev_s_doprovodem_allpass.wav', y_peakfilter, fs)

In [ ]:
from playsound import playsound

#playsound(audio)
playsound('chtit_jen_malo_zpev_s_doprovodem_allpass.wav')


In [ ]:
# plot og and filtered signal and add labels
plt.figure()
plt.plot(data)
plt.plot(y_peakfilter)
plt.xlabel('Time (samples)')
plt.ylabel('Amplitude')
plt.title('Original and Filtered Signal')
plt.legend(['Original', 'Filtered'])
plt.show()

In [ ]:
# print amplitude and faze characteristics of the filter

w, h = signal.freqz(B, A, worN=8000, fs=fs)

plt.figure(figsize=(10, 6))
plt.plot(w, 20 * np.log10(abs(h)))
plt.title('Frequency Response of the All-Pass Filter')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude (dB)')
plt.grid(which='both', linestyle='--', linewidth=0.5)
plt.axvline(fc, color='red', linestyle='--', label=f'fc = {fc} Hz')
plt.legend()
plt.show()


In [ ]:
# 1 - Az(Z) = 1 - (-c + d*(1-c)*z^-1 + z^-2)/(1+d(1-c)*z^-1 - *z^-2)
# odecteme 1 od filtru a pouzijeme tento filtr, co tento filtr dela??? Asi peak filtr?
# 1 - Az(z) = ....

# A =
# B =
# filtr = signal.lfilter(B, A, data)


In [ ]:
audio2 = './chtit jen malo zpev bez delay a reverb.wav'
data2, fs = sf.read(audio2)
data2 = data2[:10*fs]


f_ss = 44100 # Hz
# zpozdeni je 100 ms
# M...
M = 4410
g = 0.7
# y[n] = x[n] + g*x[n-M]

A = [1]
B = [1, g]
fltr = signal.lfilter(B, A, data2)

# add 4410 zeros to the start of the signal and remove 4410 samples from the end
# its 2D array so we need to add zeros to both dim
fltr = np.concatenate((np.zeros((M, 2)), fltr[:-M]), axis=0)

#sf.write('chtit_jen_malo.wav', fltr, fs)
#playsound('chtit_jen_malo.wav')


In [ ]:
# in matlab (koldovsky approach):
# output = data2 + [zeros(M, 2); fltr(1:end-M, :)]

data, fs = sf.read(audio)

data = data[:10*fs]

output = data2 + np.concatenate((np.zeros((M, 2)), data2[:-M]), axis=0)


# pridame hudbu ke zpevu -> zpev neni suchy
output = data + output


sf.write('chtit_jen_malo.wav', output, fs)
playsound('chtit_jen_malo.wav')


# v matlabu implementace pomoci filter
# y2 = filter([1 zeros(4409, 1) g], 1, data2)
# velmi spatny konvoluce pocita sileny mnozstvi nul

In [ ]:
# iir implementation:
# tohle je divny

audio2 = './chtit jen malo zpev bez delay a reverb.wav'
data2, fs = sf.read(audio2)
data2 = data2[:10*fs]

M = 4410

# y[n] = x[n] + g*y[n-M]
A = [1, -g]
B = [1]

fltr = signal.lfilter(B, A, data2)

# add 4410 zeros to the start of the signal and remove 4410 samples from the end
# its 2D array so we need to add zeros to both dim
fltr = np.concatenate((np.zeros((M, 2)), fltr[:-M]), axis=0)

sf.write('chtit_jen_malo.wav', fltr, fs)
playsound('chtit_jen_malo.wav')

In [ ]:
# zpogramovat v davkovym zpracovani ve for cyklu
# jak to udelat v pythonu na linuxu pres alsa?

In [ ]:
# implement reverb:
import soundfile as sf
import numpy as np
import scipy.signal as signal
from playsound import playsound

# y(n) = sum of tau from  0 to N (h(tau)*x(n-tau))
# LTI system
# h(n) je impulzni odezva mistonosti zavisla na rozmerech, tavru a objektech, materialu

audio_anechoic = './female_anechoic.wav'
data_anechoic, fs = sf.read(audio_anechoic)

# Create the impulse response 'h'
h = np.zeros(fs)
h[0] = 1  # direct sound
# h[1:300] remains 0

# Create a decaying reverb tail from sample 300 onward:
tail_length = fs - 300
decay = 0.0001  # Adjust this constant to control how quickly the tail decays
n = np.arange(tail_length)
# Generate random noise and multiply by an exponential decay envelope:
#h[300:] = (np.random.randn(tail_length) / 5) * np.exp(-decay * n)

# create some better decay:
# h = h.*(0.95.^(0:fs-1)/fs*50);

h[300:] = h[300:]*(0.95**(np.arange(tail_length)/fs*50))

# Filter the anechoic audio with the impulse response:
voice = signal.lfilter(h, 1, data_anechoic)

# soundsc -- sound scaling
normalized_voice = voice / np.max(np.abs(voice))

sf.write('le_voices.wav', normalized_voice, fs)
playsound('le_voices.wav')


# convoluce pouze na fir ale dopocita delku kde sparvne ma koncit, napr u ozven a dozvy (reverb
#
# )